In [2]:
require "zlib"
def show(image)
 def chunk(type, data)
   [data.bytesize, type, data, Zlib.crc32(type + data)].pack("NA4A*N")
 end
 def make_png(raw_data)
   height = raw_data.size
   width = raw_data[0].size
   s = "\x89PNG\r\n\x1a\n".force_encoding("ascii-8bit")
   s1 = chunk("IHDR", [width, height, 8, 2, 0, 0, 0].pack("NNCCCCC"))
   s2 = chunk("IDAT", Zlib::Deflate.deflate(raw_data.map {|line|
                        ([0] + line.flatten).pack("C*") }.join))
   s3 = chunk("IEND", "")
   s + s1 + s2 + s3
 end
 def to_8bit(c)
   if c.is_a?(Array)
     if c[0].is_a?(Numeric)
       c.map{|e| (e * 255).to_i }
     else
       raise "show(): bad array element " + c.to_s
     end
   else
     [(c * 255).to_i] * 3
   end
 end
 if !image.is_a?(Array)
   puts "show(): not an array" ; false
 else
   if !image[0].is_a?(Array)
     image = [image]
   end
   h = image.size
   s = h < 10 ? 20 : h < 60 ? 10 : h < 400 ? 400 / h : 1
   data = image.flat_map{|r| [r.flat_map{|c| [to_8bit(c)] * s }] * s}
   IRuby.display make_png(data), mime: "image/png" ; true
 end
end

def make2d(height, width)
  Array.new(height){ Array.new(width, 0) }
end

require 'numpy'
np = Numpy

<module 'numpy' from '/Users/nishioka/anaconda3/lib/python3.6/site-packages/numpy/__init__.py'>

## 一次元セル・オートマトン　図

In [3]:
def rule(rule,r,t)#t=隣接したセルの状態r桁2進法表記 
   def p(rule,r)#rule(10進法)の2**r桁2進法表記
    sprintf("%0*b",2**r,rule)
    "%0*b" % [2**r, rule]
   end 
    p(rule,r)[t.to_i(2)].to_i
end

:rule

In [4]:
def automaton(rule,r)
  size = 15
  image=make2d(1,size+1)
  image[0][size/2]=1
  y=1
  while image=image+make2d(1,size+1) do
    for x in 0..(size-1)
      if r%2 == 0
        if x -r/2 >= 0 && x-1+r/2 <=size-1
          image[y][x]=rule(rule,r,image[y-1][x-r/2..x-1+r/2].join)
        elsif !(x -r/2 >= 0)
          image[y][x]=rule(rule,r,(image[y-1][x-r/2+size..size-1]+image[y-1][0..x-1+r/2]).join)
        elsif !(x-1+r/2 <=size-1)
          image[y][x]=rule(rule,r,(image[y-1][x-r/2..size-1]+image[y-1][0..x-1+r/2-size]).join)
        end
      elsif r%2 == 1
        if x -(r-1)/2 >= 0 && x+(r-1)/2 <=size-1
          image[y][x]=rule(rule,r,image[y-1][x-(r-1)/2..x+(r-1)/2].join)
        elsif !(x -(r-1)/2 >= 0)
          image[y][x]=rule(rule,r,(image[y-1][x-(r-1)/2+size..size-1]+image[y-1][0..x+(r-1)/2]).join)
        elsif !(x+(r-1)/2 <=size-1)
          image[y][x]=rule(rule,r,(image[y-1][x-(r-1)/2..size-1]+image[y-1][0..x+(r-1)/2-size]).join)
        end
      end
    end
    a = 0
    while a<y
      if image[y]==image[a] then
        image[a][size]=[0,1,0]
        image[y][size]=[0,1,0]
        show(image)
        puts("周期的になるまで"+(a).to_s+"ステップ")
        puts("周期"+(y-a).to_s)
        return
      else
       a=a+1
      end
    end
    y=y+1
  end
end

:automaton

## 周期的になるまでのステップ

In [5]:
def automatona(rule,r)
  size = 15
  image=make2d(1,size+1)
  image[0][size/2]=1
  y=1
  while image=image+make2d(1,size+1) do
    for x in 0..(size-1)
      if r%2 == 0
        if x -r/2 >= 0 && x-1+r/2 <=size-1
          image[y][x]=rule(rule,r,image[y-1][x-r/2..x-1+r/2].join)
        elsif !(x -r/2 >= 0)
          image[y][x]=rule(rule,r,(image[y-1][x-r/2+size..size-1]+image[y-1][0..x-1+r/2]).join)
        elsif !(x-1+r/2 <=size-1)
          image[y][x]=rule(rule,r,(image[y-1][x-r/2..size-1]+image[y-1][0..x-1+r/2-size]).join)
        end
      elsif r%2 == 1
        if x -(r-1)/2 >= 0 && x+(r-1)/2 <=size-1
          image[y][x]=rule(rule,r,image[y-1][x-(r-1)/2..x+(r-1)/2].join)
        elsif !(x -(r-1)/2 >= 0)
          image[y][x]=rule(rule,r,(image[y-1][x-(r-1)/2+size..size-1]+image[y-1][0..x+(r-1)/2]).join)
        elsif !(x+(r-1)/2 <=size-1)
          image[y][x]=rule(rule,r,(image[y-1][x-(r-1)/2..size-1]+image[y-1][0..x+(r-1)/2-size]).join)
        end
      end
    end
    a = 0
    while a<y
      if image[y]==image[a] then
        return a
      else
       a=a+1
      end
    end
    y=y+1
  end
end
def a(r)
  g = Array.new(2**2**r)
  np = Numpy
  for i in 0..2**2**r-1
    g[i] = automatona(i,r)
  end
  np.mean(g)
end
def aa(r)
  h = Array.new(10000)
  np = Numpy
  random = Random.new
  for i in 0..9999
    h[i] = automatona(random.rand(0..2**2**r-1),r)
  end
  np.mean(h)
end
def aaa(r)
  h = Array.new(10000)
  np = Numpy
  random = Random.new
  for i in 0..9999
    h[i] = automatona(random.rand(0..2**2**r-1),r)
  end
  np.std(h)
end

:aaa

## 周期

In [6]:
def automatonb(rule,r)
  size = 15
  image=make2d(1,size+1)
  image[0][size/2]=1
  y=1
  while image=image+make2d(1,size+1) do
    for x in 0..(size-1)
      if r%2 == 0
        if x -r/2 >= 0 && x-1+r/2 <=size-1
          image[y][x]=rule(rule,r,image[y-1][x-r/2..x-1+r/2].join)
        elsif !(x -r/2 >= 0)
          image[y][x]=rule(rule,r,(image[y-1][x-r/2+size..size-1]+image[y-1][0..x-1+r/2]).join)
        elsif !(x-1+r/2 <=size-1)
          image[y][x]=rule(rule,r,(image[y-1][x-r/2..size-1]+image[y-1][0..x-1+r/2-size]).join)
        end
      elsif r%2 == 1
        if x -(r-1)/2 >= 0 && x+(r-1)/2 <=size-1
          image[y][x]=rule(rule,r,image[y-1][x-(r-1)/2..x+(r-1)/2].join)
        elsif !(x -(r-1)/2 >= 0)
          image[y][x]=rule(rule,r,(image[y-1][x-(r-1)/2+size..size-1]+image[y-1][0..x+(r-1)/2]).join)
        elsif !(x+(r-1)/2 <=size-1)
          image[y][x]=rule(rule,r,(image[y-1][x-(r-1)/2..size-1]+image[y-1][0..x+(r-1)/2-size]).join)
        end
      end
    end
    a = 0
    while a<y
      if image[y]==image[a] then
        return y-a
      else
       a=a+1
      end
    end
    y=y+1
  end
end
def b(r)
  h = Array.new(2**2**r)
  np = Numpy
  for i in 0..2**2**r-1
    h[i] = automatonb(i,r)
  end
  np.mean(h)
end
def bb(r)
  h = Array.new(10000)
  np = Numpy
  random = Random.new
  for i in 0..9999
    h[i] = automatonb(random.rand(0..2**2**r-1),r)
  end
  np.mean(h)
end

:bb

In [12]:
automaton(135,3)

周期的になるまで6ステップ
周期450
